# ULX Population Estimate

The starting point for the project is the simulation files from STARTRACK provided by Grzegorz Wiktorowicz.

The files have the following naming structure: 'Z_0.002_tage_20.dat' and identify both the Metallicity and the Age of system (time since ZAMS in Myr).

In [10]:
ls

test.py                  Z_0.002_tage_10000.dat  Z_0.02_tage_1000.dat
Z_0.0002_tage_10000.dat  Z_0.002_tage_1000.dat   Z_0.02_tage_100.dat
Z_0.0002_tage_1000.dat   Z_0.002_tage_100.dat    Z_0.02_tage_10.dat
Z_0.0002_tage_100.dat    Z_0.002_tage_10.dat     Z_0.02_tage_2000.dat
Z_0.0002_tage_10.dat     Z_0.002_tage_2000.dat   Z_0.02_tage_200.dat
Z_0.0002_tage_2000.dat   Z_0.002_tage_200.dat    Z_0.02_tage_20.dat
Z_0.0002_tage_200.dat    Z_0.002_tage_20.dat     Z_0.02_tage_5000.dat
Z_0.0002_tage_20.dat     Z_0.002_tage_5000.dat   Z_0.02_tage_500.dat
Z_0.0002_tage_5000.dat   Z_0.002_tage_500.dat    Z_0.02_tage_50.dat
Z_0.0002_tage_500.dat    Z_0.002_tage_50.dat
Z_0.0002_tage_50.dat     Z_0.02_tage_10000.dat


Each file contains 4 columns:

    mdot:
        mass transfer rate (in solar masses per Myr; to the disk, not to the accretor, 
        which is lower due to the mass loss in disk wind),
    
    m:
        mass of a compact object (in solar masses; we assumed that these with a mass 
        lower than 2.5 Msun are NSs, whereas the heavier ones are BHs).
    
    idum, iidd:
        identificators of the system in the simulations. They have no physical meaning,
        but will allow to find more info about some potentially odd, or interesting system.
        

We may import them via the following:

In [15]:
import pandas as pd
import glob
import numpy as np
def ImportFiles():
    df_dict = {}  #Dictionary for storing dataframes for m_dot
    files = glob.glob('*.dat')

    for filename in files:
        # print(filename)
        df_dict[filename] = pd.read_csv(filename, delim_whitespace=True,
               header=None, names=['mdot', 'm', 'idum', 'iidd'])
    
        cut_string = filename.split(sep='_', maxsplit=4)
        df_dict[filename]['Z'] = float(cut_string[1])
        df_dict[filename]['tage'] = float(cut_string[3][:-4])
        
    
    # Concat all the datafames into one.
    df_master = pd.concat(df_dict, ignore_index=True)
    df_master = df_master.drop(['idum'], axis=1) #Drop useless columns
    df_master = df_master.drop(['iidd'], axis=1) #Drop useless columns
    df_master['is_bh'] = np.where(df_master['m'] < 2.5, 0 , 1) #Add type column
    return df_master

df_master = ImportFiles()

In [23]:
df_master.head()

,mdot,m,Z,tage,is_bh
0,3.90,25.9,0.0002,10.0,1
1,1.70,17.0,0.0002,10.0,1
2,2.70,13.1,0.0002,10.0,1
3,2.60,25.8,0.0002,10.0,1
4,0.65,15.8,0.0002,10.0,1


In [32]:
df_master['m'].describe()

count    36420.000000
mean         2.103177
std          3.049861
min          1.100000
25%          1.400000
50%          1.500000
75%          1.600000
max         44.300000
Name: m, dtype: float64

In [33]:
df_master['mdot'].describe()

count    36420.000000
mean         0.335150
std         14.541906
min          0.000001
25%          0.000004
50%          0.000011
75%          0.000097
max       1300.000000
Name: mdot, dtype: float64

## Calculation of physical quantities

The next thing to do is calculate various quantities from the data such as the maximum luminosity, beaming factor etc.

### Mass accretion rate in grams/second: $\dot{m}_{gs}$
\begin{equation}
    \dot{m}_{gs} = \dot{m} \frac{M_{\odot}}{\mathrm{Myr}}
\end{equation}
### Eddington Luminosity:
\begin{equation}
    L_{Edd} = 1.2 \times 10 ^{38} m\\
\end{equation}
### Eddington Mass:
\begin{equation}
    \dot{m}_{Edd} = \frac{ L_{Edd} } { \eta c^{2} }\\
\end{equation}
### Eddington Ratio:
\begin{equation}
    \dot{m}_{0} = \frac{\dot{m}}{\dot{m}_{Edd}} \\
\end{equation}
### Isotropic Luminosity:
\begin{equation}
L_{xtot} =
\left\{
	\begin{array}{ll}
		L_{Edd} (1 + \mathrm{log}(\dot{m}_{0}))  & \mbox{if } \dot{m}_{0} \geq 1 \\
		L_{Edd} \dot{m}_{0} & \mbox{otherwise}
	\end{array}
\right.
\end{equation}
### Beaming Factor:
\begin{equation}
b =
\left\{
	\begin{array}{ll}
		1  & \mbox{if } \dot{m}_{0} < 8.5 \\
		73/\dot{m}_{0}^{2} & \mbox{if } \dot{m}_{0} \geq 8.5 \\
		3.2 \times 10^{-3} & \mbox{if } \dot{m}_{0} > 150 \\
	\end{array}
\right.
\end{equation}
### Maximum Beamed Luminosity:
\begin{equation}
    L_{x} = \frac{ L_{xtot} } { b }\\
	\label{eq:Lx}
\end{equation}
### Eddington Ratios for beamed and unbeamed:
\begin{equation}
    \mathrm{ratio} = \frac{ L_{xtot} } { L_{Edd} }\\
    \mathrm{ratio_{beamed}} = \frac{ L_{x} } { L_{Edd} }\\
\end{equation}
### Full Opening Angle:
\begin{equation}
    b = 1-\mathrm{cos}(\theta/2)\\
    \therefore \theta = 2\mathrm{arccos}(1-b)\\
\end{equation}
### Wind Opening Angle:
\begin{equation}
\zeta = \mathrm{tan}\left [ \frac{\pi}{2} - \mathrm{acos} \left (  1 - \frac{73}{\dot{m_{0}}^{2}}   \right )   \right ]
	\label{eq:}
\end{equation}
### Schwarzchild Radius (cm):
\begin{equation}
R_{s} = \frac{2GM_{1}}{c^{2}}
\end{equation}
### Spherization Radius:
\begin{equation}
R_{sph} \approx 27 \frac{ \dot{M} R_{s}} {4 \dot{M_{E}} }
\end{equation}
